<a href="https://colab.research.google.com/github/amiholz/NN/blob/master/Ex1_q3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
try:
  %tensorflow_version 2.x
except Exception:
  pass

TensorFlow 2.x selected.


In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras import Model

mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# Add a channels dimension

x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]

train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(10000).batch(32)


test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

class MyModel(Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.conv = Conv2D(2, 5, activation='relu')
        self.flatten = Flatten()
        self.d = Dense(10, activation='softmax')
    
    def call(self, x):
        x = self.conv(x)
        x = self.flatten(x)
        return self.d(x)



11493376/11490434 [==============================] - 0s 0us/step


In [0]:
# Create an instance of the model
model = MyModel()


loss_object = tf.keras.losses.SparseCategoricalCrossentropy()

optimizer = tf.keras.optimizers.Adam()


train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

@tf.function
def train_step(images, labels):
    
    with tf.GradientTape() as tape:
        predictions = model(images)
        loss = loss_object(labels, predictions)
        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    
    train_loss(loss)
    train_accuracy(labels, predictions)

@tf.function
def test_step(images, labels):
    predictions = model(images)
    t_loss = loss_object(labels, predictions)
    
    test_loss(t_loss)
    test_accuracy(labels, predictions)



In [6]:
EPOCHS = 5

for epoch in range(EPOCHS):
    for images, labels in train_ds:
        train_step(images, labels)

    for (test_images, test_labels) in test_ds:
        test_step(test_images, test_labels)
        template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
    print(template.format(epoch+1,
                          train_loss.result(),
                          train_accuracy.result()*100,
                          test_loss.result(),
                          test_accuracy.result()*100))
    # test_accuracies.append()
    print(" -- ", flush=True)
    
    # Reset the metrics for the next epoch
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()

Epoch 1, Loss: 0.09881013631820679, Accuracy: 96.94166564941406, Test Loss: 0.10779592394828796, Test Accuracy: 96.37000274658203
 -- 
Epoch 2, Loss: 0.09186570346355438, Accuracy: 97.09333038330078, Test Loss: 0.1116868406534195, Test Accuracy: 96.36000061035156
 -- 
Epoch 3, Loss: 0.08754085749387741, Accuracy: 97.30500030517578, Test Loss: 0.10614634305238724, Test Accuracy: 96.59000396728516
 -- 
Epoch 4, Loss: 0.08585019409656525, Accuracy: 97.29666900634766, Test Loss: 0.10220541059970856, Test Accuracy: 96.66000366210938
 -- 
Epoch 5, Loss: 0.08266407996416092, Accuracy: 97.43000030517578, Test Loss: 0.12273678928613663, Test Accuracy: 96.34000396728516
 -- 
